In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify, scale_data, ord_encoding, initiate_data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)


In [3]:
## Set up regression model to be used in the function
from sklearn.linear_model import Lasso
lasso = Lasso()
alphaRange = np.linspace(0,0.0005,5)
params = {'alpha': alphaRange, 'max_iter' : [10000]}
gs = GridSearchCV(lasso, params, scoring = 'r2', cv=5)
gs.fit(train_data_linear, train_target_linear)

C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.273316087358635, tolerance: 0.020321695490145075
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimat

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([0.      , 0.000125, 0.00025 , 0.000375, 0.0005  ]),
                         'max_iter': [10000]},
             scoring='r2')

In [13]:
import tabpy_client
from tabpy.tabpy_tools.client import Client

def predict_for_tableau(_arg1):
    
    # _arg1 = "OldTown"
    from data_preprocessing_final import initiate_data
    housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)

    row = housing[housing['PID'] == 1].copy()
    row['Neighborhood'] = _arg1



    housing.drop(housing[housing['PID'] == 1].index, inplace=True)
    housing = housing.append(row)

    ## Set up train test data
    ## note, random seed is set so that PID == 1 will always be in test daata
    train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)

    ## use the model to predict on the test set and then pull out where PID == 1
    predict = gs.predict(test_data_linear)
    predict = pd.DataFrame(predict)
    predict.columns = ['Prediction']
    final = pd.concat([predict, test_data_linear, test_target_linear], axis =1)
    log_price = final[final['PID'] == 1]['Prediction']
    price = np.exp(log_price)
    return price


# client = tabpy_client.Client('http://localhost:9004/')
# client.deploy('predict_for_tableau', predict_for_tableau, 'predicts price based on inputs', override = True)